In [102]:
##Code in this file has been modified from Boris Murmann's github below, and the open-source textbook from Harald Pretl and colleagues:
# https://github.com/bmurmann/Book-on-gm-ID-design/blob/main/starter_files_open_source_tools/gf180mcuD/techsweep_plots_from_mat.ipynb
# Murmann and his colleague have written a book on gm/id-based design, "Systematic Design of Analog CMOS Circuits" (2017)

#Pretl's Open-Source Book: https://iic-jku.github.io/analog-circuit-design/analog_circuit_design.pdf

# Copyright 2024 Harald Pretl
# Licensed under the Apache License, Version 2.0 (the “License”); you may not use this
# file except in compliance with the License. You may obtain a copy of the License at
# http://www.apache.org/licenses/LICENSE-2.0


import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
from pygmid import Lookup as lk

nfet = lk('../nfet_03v3.mat')
pfet = lk('../pfet_03v3.mat')
VDS1 = 1.65

In [103]:
# define the given parameters as taken from the specification table or inital guesses
c_load = 2e-12 #Expected given parasitics + breadboard mounting of device
gm_id_m12 = 12
gm_id_m34 = 8
l_12 = 1
l_34 = 1

# Current Source Limitations
i_bias_in = 10e-6               #Reference current is 10uA. Note that if some internal design uses something like 30uA, you can mirror that value too
w_m6 = 3.5                      #UNIT NMOS WIDTH
l_56 = 1                        #UNIT NMOS LENGTH

f_bw = 800e3 # -3dB bandwidth of the voltage buffer
i_total_limit = 50e-6
i_bias_in = 10e-6
output_voltage = 1.65
vin_min = 1
vin_max = 2.8
vdd_min = 3.2
vdd_max = 3.4

In [104]:
# we get the required gm of M1/2 from the -3dB bandwidth requirement of the voltage buffer
# we add a factor of 3 to allow for PVT variation plus additional MOSFET parasitic loading
gm_m12 = f_bw * 3 * 4*np.pi*c_load 
print('gm12 =', round(gm_m12/1e-3, 4), 'mS')

# since we know gm12 and the gmid we can calculate the bias current
id_m12 = gm_m12 / gm_id_m12
i_total = 2*id_m12
print('i_total (exact) =', round(i_total/1e-6, 1), 'µA')

# we round to 0.5µA bias currents
i_total = max(round(i_total / 1e-6 * 2) / 2 * 1e-6, 0.5e-6)
id_m12 = i_total/2
print('i_total (rounded) =', i_total/1e-6, 'µA')
if i_total < i_total_limit:
    print('[info] power consumption target is met!')
else:
    print('[info] power consumption target is NOT met!')


gm12 = 0.0603 mS
i_total (exact) = 10.1 µA
i_total (rounded) = 10.0 µA
[info] power consumption target is met!


In [105]:
# we calculate the dc gain

#Intrinsic gain of mirrors and diff pair
gm_gds_m12 = nfet.lookup('GM_GDS', GM_ID=gm_id_m12, L=l_12, VDS=VDS1, VSB=0)
gm_gds_m34 = pfet.lookup('GM_GDS', GM_ID=gm_id_m34, L=l_34, VDS=VDS1, VSB=0)

#get diff pair gds values
gds_m12 = gm_m12 / gm_gds_m12

#get current mirror gds, gm
gm_m34 = gm_id_m34 * i_total/2
gds_m34 = gm_m34 / gm_gds_m34

a0 = gm_m12 / (gds_m12 + gds_m34)
print('a0 =', round(20*np.log10(a0), 1), 'dB')


a0 = 48.4 dB


In [106]:
# we calculate the MOSFET capacitance which adds to Cload, to see the impact on the BW
gm_cgs_m12 = nfet.lookup('GM_CGS', GM_ID=gm_id_m12, L=l_12, VDS=VDS1, VSB=0)
gm_cdd_m12 = nfet.lookup('GM_CDD', GM_ID=gm_id_m12, L=l_12, VDS=VDS1, VSB=0)
gm_cdd_m34 = pfet.lookup('GM_CDD', GM_ID=gm_id_m34, L=l_34, VDS=VDS1, VSB=0)
c_load_parasitic = abs(gm_m12/gm_cgs_m12) + abs(gm_m12/gm_cdd_m12) + abs(gm_m34/gm_cdd_m34)
print('additional load capacitance =', round(c_load_parasitic/1e-15, 1), 'fF')
f_bw = gm_m12 / (4*np.pi * (c_load + c_load_parasitic))
print('unity gain bandwidth incl. parasitics =', round(f_bw/1e6, 2), 'MHz')

additional load capacitance = 14.9 fF
unity gain bandwidth incl. parasitics = 2.38 MHz


In [107]:
vgs_m12 = nfet.look_upVGS(GM_ID=gm_id_m12, L=l_12, VDS=VDS1, VSB=0.0)
vgs_m34 = pfet.look_upVGS(GM_ID=gm_id_m34, L=l_34, VDS=VDS1, VSB=0.0)
print('vgs_12 =', round(float(vgs_m12), 3), 'V')
print('vgs_34 =', round(float(vgs_m34), 3), 'V')




vgs_12 = 0.789 V
vgs_34 = 0.999 V


In [108]:
# calculate settling time due to slewing with the calculated bias current
t_slew = (c_load + c_load_parasitic) * output_voltage / i_total
print('slewing time =', round(t_slew/1e-6, 3), 'µs')
t_settle = 5/(2*np.pi*f_bw)
print('settling time =', round(t_settle/1e-6, 3), 'µs')

slewing time = 0.332 µs
settling time = 0.334 µs


In [109]:
# calculate voltage gain error
gain_error = a0 / (1 + a0)
print('voltage gain error =', round((gain_error-1)*100, 1), '%')

voltage gain error = -0.4 %


In [110]:
# calculate total rms output noise
sth_m12 = nfet.lookup('STH_GM', VGS=vgs_m12, L=l_12, VDS=VDS1, VSB=0) * gm_m12
gamma_m12 = sth_m12/(4*1.38e-23*300*gm_m12)
sth_m34 = pfet.lookup('STH_GM', VGS=vgs_m34, L=l_34, VDS=VDS1, VSB=0) * gm_m34
gamma_m34 = sth_m34/(4*1.38e-23*300*gm_m34)
output_noise_rms = np.sqrt(1.38e-23*300 / (c_load + c_load_parasitic) * (2*gamma_m12 + 2*gamma_m34*gm_m34/gm_m12))
print('output noise =', round(output_noise_rms/1e-6, 1), 'µVrms')


output noise = 73.1 µVrms


In [111]:

# calculate all widths
id_w_m12 = nfet.lookup('ID_W', GM_ID=gm_id_m12, L=l_12, VDS=vgs_m12, VSB=0)
w_12 = id_m12 / id_w_m12
w_12_round = max(round(w_12*2)/2, 0.5)
print('M1/2 W =', round(w_12, 2), 'um, rounded W =', w_12_round, 'um')
id_m34 = id_m12
id_w_m34 = pfet.lookup('ID_W', GM_ID=gm_id_m34, L=l_34, VDS=vgs_m34, VSB=0)
w_34 = id_m34 / id_w_m34
w_34_round = max(round(w_34*2)/2, 0.5)
print('M3/4 W =', round(w_34, 2), 'um, rounded W =', w_34_round, 'um')

#M5,6
w_ratio_m5 = (i_total / i_bias_in) 
id_w_m5 = i_total/(w_ratio_m5 * w_m6)
id_w_m6 = i_bias_in/w_m6
vgs_m56 = pfet.look_upVGS(ID_W=id_w_m5, L=l_56, VDS=VDS1, VSB=0.0)


w_5 = (w_ratio_m5 * w_m6)
w_5_round = max(round(w_5*2)/2, 0.5)
print('M5 W =', round(w_5, 2), 'um, rounded W =', w_5_round, 'um')

w_6_round = max(round(w_m6*2)/2, 0.5)
print('M6 W =', round(w_6_round, 2), 'um')

print(">>  [info] M6 was assumed to be a unit transistor. You MUST verify that the conclusions are reasonable:")

print("M5 W requires", round(w_ratio_m5,4), "unit transistors. Can you achieve this ratio?!")
print('vgs_56 =', round(float(vgs_m56), 3), 'V')

gm_id_5 = pfet.lookup('GM_ID', ID_W=id_w_m5, L=l_56, VDS=vgs_m56, VSB=0)
gm_id_5 = pfet.lookup('GM_ID', ID_W=id_w_m6, L=l_56, VDS=vgs_m56, VSB=0)
print("M5 GM/ID =", round(1*gm_id_5, 3), 'S/A')
print("M6 GM/ID =", round(1*gm_id_5, 3), 'S/A')


M1/2 W = 3.64 um, rounded W = 3.5 um
M3/4 W = 5.84 um, rounded W = 6.0 um
M5 W = 3.5 um, rounded W = 3.5 um
M6 W = 3.5 um
>>  [info] M6 was assumed to be a unit transistor. You MUST verify that the conclusions are reasonable:
M5 W requires 1.0 unit transistors. Can you achieve this ratio?!
vgs_56 = 1.202 V
M5 GM/ID = 4.356 S/A
M6 GM/ID = 4.356 S/A


In [112]:
# Estimate current mirror pole positions
